# 生成式網絡

循環神經網絡（RNNs）及其門控單元變體，例如長短期記憶單元（LSTMs）和門控循環單元（GRUs），提供了一種語言建模的機制，也就是說，它們可以學習詞語的排列順序，並對序列中的下一個詞進行預測。這使得我們可以使用 RNNs 進行**生成任務**，例如普通文本生成、機器翻譯，甚至是圖像描述。

在上一單元中討論的 RNN 架構中，每個 RNN 單元都會生成下一個隱藏狀態作為輸出。然而，我們也可以為每個循環單元添加另一個輸出，這樣就可以輸出一個**序列**（其長度與原始序列相等）。此外，我們還可以使用不在每一步接受輸入的 RNN 單元，而僅僅接受一些初始狀態向量，然後生成一系列的輸出。

在這份筆記中，我們將專注於幫助我們生成文本的簡單生成模型。為了簡化，我們將構建**字元級網絡**，逐字生成文本。在訓練過程中，我們需要採用一些文本語料庫，並將其拆分為字元序列。


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## 建立字元詞彙表

為了建立字元級別的生成網絡，我們需要將文本拆分為單個字元，而不是單詞。我們之前使用的 `TextVectorization` 層無法做到這一點，因此我們有以下兩個選擇：

* 手動載入文本並自行進行分詞，如[這個官方 Keras 範例](https://keras.io/examples/generative/lstm_character_level_text_generation/)中所示
* 使用 `Tokenizer` 類進行字元級別的分詞。

我們將選擇第二種方法。`Tokenizer` 也可以用於將文本分詞為單詞，因此可以很輕鬆地從字元級分詞切換到單詞級分詞。

要進行字元級分詞，我們需要傳遞參數 `char_level=True`：


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

我們也希望使用一個特殊的標記來表示**序列結束**，我們將其稱為 `<eos>`。讓我們手動將其添加到詞彙表中：


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## 訓練生成式 RNN 來生成標題

我們將以以下方式訓練 RNN 來生成新聞標題。在每一步中，我們會取一個標題，將其輸入到 RNN 中，並對於每個輸入的字元，要求網路生成下一個輸出的字元：

![顯示 RNN 生成單詞 'HELLO' 的範例圖片。](../../../../../translated_images/zh-MO/rnn-generate.56c54afb52f9781d.webp)

對於序列中的最後一個字元，我們會要求網路生成 `<eos>` 標記。

這裡使用的生成式 RNN 與其他的主要區別在於，我們會從 RNN 的每一步輸出中取結果，而不僅僅是從最後一個單元格中取結果。這可以通過為 RNN 單元指定 `return_sequences` 參數來實現。

因此，在訓練過程中，網路的輸入將是一個特定長度的編碼字元序列，而輸出則是一個相同長度的序列，但向後偏移一個元素並以 `<eos>` 結尾。小批次（minibatch）將由多個這樣的序列組成，我們需要使用**填充（padding）**來對齊所有序列。

接下來，我們來建立一些函數，用於轉換數據集。由於我們希望在小批次層級進行序列填充，我們會先通過調用 `.batch()` 將數據集分批，然後使用 `map` 來進行轉換。因此，轉換函數將以整個小批次作為參數：


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

以下是我們在這裡執行的一些重要步驟：
* 我們首先從字串張量中提取實際文本
* `text_to_sequences` 將字串列表轉換為整數張量列表
* `pad_sequences` 將這些張量填充到它們的最大長度
* 最後，我們對所有字符進行獨熱編碼，並執行移位和 `<eos>` 附加操作。我們很快就會了解為什麼需要使用獨熱編碼的字符

然而，這個函數是 **Pythonic** 的，也就是說，它無法自動轉換為 Tensorflow 的計算圖。如果我們直接在 `Dataset.map` 函數中使用這個函數，會出現錯誤。我們需要使用 `py_function` 包裝器來封裝這個 Pythonic 調用：


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **注意**：區分 Pythonic 和 Tensorflow 的轉換函數可能看起來有點複雜，你可能會疑惑為什麼我們不在將資料傳遞給 `fit` 之前，使用標準的 Python 函數來轉換資料。雖然這確實是可行的，但使用 `Dataset.map` 有一個巨大的優勢，因為資料轉換管道是使用 Tensorflow 的計算圖執行的，這可以利用 GPU 的計算能力，並且減少在 CPU 和 GPU 之間傳遞資料的需求。

現在我們可以建立生成器網路並開始訓練。它可以基於我們在上一單元中討論過的任何循環單元（簡單的、LSTM 或 GRU）。在我們的例子中，我們將使用 LSTM。

由於網路以字元作為輸入，且詞彙表的大小相對較小，我們不需要嵌入層，直接將 one-hot 編碼的輸入傳遞給 LSTM 單元即可。輸出層將是一個 `Dense` 分類器，它會將 LSTM 的輸出轉換為 one-hot 編碼的標記編號。

此外，因為我們處理的是可變長度的序列，我們可以使用 `Masking` 層來建立一個遮罩，忽略字串中填充的部分。這並不是絕對必要的，因為我們對超過 `<eos>` 標記的部分並不特別感興趣，但我們會使用它來獲得一些使用這類型層的經驗。`input_shape` 將是 `(None, vocab_size)`，其中 `None` 表示可變長度的序列，而輸出形狀也是 `(None, vocab_size)`，正如你可以從 `summary` 中看到的那樣。


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## 生成輸出

現在我們已經訓練了模型，接下來我們想要使用它來生成一些輸出。首先，我們需要一種方法來解碼由一系列標記數字表示的文本。為此，我們可以使用 `tokenizer.sequences_to_texts` 函數；然而，這個方法在字元級別的標記化中效果並不好。因此，我們將從 tokenizer 中取得一個標記的字典（稱為 `word_index`），建立一個反向映射，並撰寫我們自己的解碼函數：


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

現在，我們開始進行生成。我們將以某個字串 `start` 作為起點，將其編碼成一個序列 `inp`，然後在每一步中，我們會呼叫網路來推斷下一個字元。

網路的輸出 `out` 是一個包含 `vocab_size` 元素的向量，代表每個標記的概率。我們可以使用 `argmax` 找出最可能的標記編號。接著，我們將這個字元附加到已生成的標記列表中，並繼續進行生成。這個生成一個字元的過程會重複執行 `size` 次，以生成所需的字元數量。如果在過程中遇到 `eos_token`，我們會提前終止生成。


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## 在訓練期間抽樣輸出

由於我們沒有任何像 *準確率* 這樣的有用指標，我們唯一能看到模型是否有所改進的方法就是在訓練期間通過 **抽樣** 生成的字串來進行檢查。為了實現這一點，我們將使用 **回調函數**，也就是可以傳遞給 `fit` 函數並在訓練期間定期被調用的函數。


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


這個範例已經生成了一些相當不錯的文本，但仍有多種方式可以進一步改進：

* **更多文本**。我們僅使用了標題來完成任務，但您可能希望嘗試使用完整的文本。請記住，RNN在處理長序列方面表現不佳，因此可以將文本拆分成較短的句子，或者始終以固定的序列長度進行訓練，例如預定義的值 `num_chars`（例如 256）。您可以嘗試將上述範例改造成這樣的架構，並參考 [官方 Keras 教學](https://keras.io/examples/generative/lstm_character_level_text_generation/) 作為靈感。

* **多層 LSTM**。嘗試使用 2 或 3 層的 LSTM 單元是有意義的。如我們在前一單元提到的，每層 LSTM 都會從文本中提取特定的模式，而在字元級生成器的情況下，我們可以預期較低層的 LSTM 負責提取音節，而較高層則負責提取單詞及單詞組合。這可以通過向 LSTM 構造函數傳遞層數參數來簡單實現。

* 您也可以嘗試使用 **GRU 單元**，看看哪種表現更好，並嘗試 **不同的隱藏層大小**。隱藏層過大可能導致過度擬合（例如，網絡會學習精確的文本），而過小的大小可能無法生成良好的結果。


## 軟性文本生成與溫度

在之前 `generate` 的定義中，我們總是選擇機率最高的字元作為生成文本的下一個字元。這導致生成的文本經常在相同的字元序列之間不斷循環，例如以下例子：
```
today of the second the company and a second the company ...
```

然而，如果我們觀察下一個字元的機率分佈，可能會發現幾個最高機率之間的差距並不大，例如一個字元的機率是 0.2，另一個是 0.19，等等。例如，在尋找序列 *play* 的下一個字元時，下一個字元可能同樣有可能是空格，或者是 **e**（如單字 *player* 中的情況）。

這讓我們得出一個結論：選擇機率最高的字元並不總是「公平」的，因為選擇第二高的字元也可能生成有意義的文本。更明智的做法是根據網路輸出的機率分佈來**抽樣**字元。

這種抽樣可以使用 `np.multinomial` 函數來完成，該函數實現了所謂的**多項分佈**。以下是一個實現這種**軟性**文本生成的函數：


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

我們引入了一個名為 **溫度** 的參數，用於指示我們應該多麼嚴格地遵循最高概率。如果溫度為 1.0，我們進行公平的多項式抽樣，而當溫度趨於無窮大時——所有概率變得相等，我們隨機選擇下一個字符。在下面的例子中，我們可以觀察到當我們將溫度增加得太高時，文本變得毫無意義，而當溫度接近 0 時，它則類似於「循環」的硬生成文本。



---

**免責聲明**：  
本文件使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。應以原始語言的文件作為權威來源。對於關鍵資訊，建議尋求專業人工翻譯。我們對於因使用本翻譯而引起的任何誤解或錯誤解讀概不負責。
